In [17]:
#import Pkg; Pkg.add("Pycall")



In [28]:
using PyCall

using LowRankModels
using CSV
using DataFrames
using Pkg
using StatsBase
using Random, SparseArrays

using Random
using LinearAlgebra
using DataFrames             # Data tables are called "DataFrames"
using StatsPlots             # load plotting packages 
using Statistics             # basic statistical functions
using CSV  
using SparseArrays
using Pkg
using StatsBase
pyplot()

Plots.PyPlotBackend()

In [29]:
df2 = CSV.read("../data/NYC_SPARCS_cancer.csv")
df = CSV.read("../data/SPARCS_cancer_binary.csv")
dfk = CSV.read("../data/SPARCS_kidney_cancer_binary.csv")



,Column1,Total_Charges,APR_Severity_of_Illness_Code,APR_Risk_of_Mortality,Age_Ordinal
,Int64,Float64,Int64,Int64,Int64
1,0,20601.5,2,4,3
2,1,12020.9,1,1,5
3,2,40220.4,2,1,4
4,3,9914.54,1,1,4
5,4,10182.1,2,1,4
6,5,9090.09,2,1,4
7,6,28436.7,2,1,4
8,7,6183.3,2,1,3
9,8,6342.01,2,1,2


In [30]:
for name in names(dfk)
    println(":", name, ", ")
end

:Column1, 
:Total_Charges, 
:APR_Severity_of_Illness_Code, 
:APR_Risk_of_Mortality, 
:Age_Ordinal, 
:Length_of_Stay, 
:Health_Service_Area_New York City, 
:Hospital_County_Bronx, 
:Hospital_County_Kings, 
:Hospital_County_Manhattan, 
:Hospital_County_Queens, 
:Hospital_County_Richmond, 
:Type_of_Admission_Elective, 
:Type_of_Admission_Emergency, 
:Type_of_Admission_Urgent, 
:Type_of_Admission_Not Available, 
:Patient_Disposition_Home or Self Care, 
:Patient_Disposition_Medicare Cert Long Term Care Hospital, 
:Patient_Disposition_Home w/ Home Health Services, 
:Patient_Disposition_Expired, 
:Patient_Disposition_Skilled Nursing Home, 
:Patient_Disposition_Short-term Hospital, 
:Patient_Disposition_Hospice - Medical Facility, 
:Patient_Disposition_Facility w/ Custodial/Supportive Care, 
:Patient_Disposition_Left Against Medical Advice, 
:Patient_Disposition_Inpatient Rehabilitation Facility, 
:Patient_Disposition_Hospice - Home, 
:Patient_Disposition_Psychiatric Hospital or Unit of Hosp, 

In [31]:
features = [
    :Total_Charges,
    :APR_Severity_of_Illness_Code,
    :APR_Risk_of_Mortality,
    :Age_Ordinal,
    :Length_of_Stay, 
    #:Gender_F
]

#df = df[:,[1 2]]
#df = select!(df, Not(:B))

5-element Array{Symbol,1}:
 :Total_Charges
 :APR_Severity_of_Illness_Code
 :APR_Risk_of_Mortality
 :Age_Ordinal
 :Length_of_Stay

In [32]:
dfsmall1 = df[features]

#dfsmall1 = dfk[features]

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[32]:1
└ @ Core In[32]:1


,Total_Charges,APR_Severity_of_Illness_Code,APR_Risk_of_Mortality,Age_Ordinal
,Float64,Int64,Int64,Int64
1,216494.0,3,4,3
2,194331.0,4,3,5
3,79396.5,3,2,4
4,86736.8,3,4,4
5,178593.0,3,1,1
6,57239.2,2,4,4
7,111640.0,4,2,4
8,31801.9,2,4,4
9,38714.5,3,2,5


# kmeans k=4 quadloss

In [33]:
k = 4
A= dfsmall1
losses = QuadLoss() # minimize squared distance to cluster centroids
rx = UnitOneSparseConstraint() # each row is assigned to exactly one cluster
ry = ZeroReg() # no regularization on the cluster centroids
glrm = GLRM(A,losses,rx,ry,k)

GLRM(35804×5 DataFrame. Omitted printing of 2 columns
│ Row   │ Total_Charges │ APR_Severity_of_Illness_Code │ APR_Risk_of_Mortality │
│       │ Float64       │ Int64                        │ Int64                 │
├───────┼───────────────┼──────────────────────────────┼───────────────────────┤
│ 1     │ 216494.0      │ 3                            │ 4                     │
│ 2     │ 194331.0      │ 4                            │ 3                     │
│ 3     │ 79396.5       │ 3                            │ 2                     │
│ 4     │ 86736.8       │ 3                            │ 4                     │
│ 5     │ 178593.0      │ 3                            │ 1                     │
│ 6     │ 57239.2       │ 2                            │ 4                     │
│ 7     │ 111640.0      │ 4                            │ 2                     │
│ 8     │ 31801.9       │ 2                            │ 4                     │
│ 9     │ 38714.5       │ 3                            

# init_kmeans debugged

In [34]:
m,n = size(glrm.A)
k = glrm.k
possible_centers = Set(1:m)
glrm.Y = randn(k,n)
# assign first center randomly
i = sample(1:m)
setdiff!(possible_centers, i)

glrm.Y[1,glrm.observed_features[i]] =  vec(convert(Array, glrm.A[i,glrm.observed_features[i]]))
#glrm.Y[1,glrm.observed_features[i]] = convert(Matrix, glrm.A[i,glrm.observed_features[i]])
# assign next centers one by one
for l=1:k-1
min_dists_per_obs = zeros(m)
for i in possible_centers
    d = zeros(l)
    for j in glrm.observed_features[i]
        for ll=1:l
            d[ll] += evaluate(glrm.losses[j], glrm.Y[ll,j], glrm.A[i,j])
        end
    end
    min_dists_per_obs[i] = minimum(d)/length(glrm.observed_features[i])
end
furthest_index = wsample(1:m,min_dists_per_obs)

glrm.Y[l+1,glrm.observed_features[furthest_index]] = vec(convert(Array, glrm.A[furthest_index,glrm.observed_features[furthest_index]]))
end
return glrm    


┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[34]:8
└ @ Core In[34]:8
┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[34]:25
└ @ Core ./In[34]:25


GLRM(35804×5 DataFrame. Omitted printing of 2 columns
│ Row   │ Total_Charges │ APR_Severity_of_Illness_Code │ APR_Risk_of_Mortality │
│       │ Float64       │ Int64                        │ Int64                 │
├───────┼───────────────┼──────────────────────────────┼───────────────────────┤
│ 1     │ 216494.0      │ 3                            │ 4                     │
│ 2     │ 194331.0      │ 4                            │ 3                     │
│ 3     │ 79396.5       │ 3                            │ 2                     │
│ 4     │ 86736.8       │ 3                            │ 4                     │
│ 5     │ 178593.0      │ 3                            │ 1                     │
│ 6     │ 57239.2       │ 2                            │ 4                     │
│ 7     │ 111640.0      │ 4                            │ 2                     │
│ 8     │ 31801.9       │ 2                            │ 4                     │
│ 9     │ 38714.5       │ 3                            

In [35]:
#init_kmeanspp!(glrm)

In [36]:
X,Y,ch = fit!(glrm)

Fitting GLRM
Iteration 10: objective value = 9.04964065895266e13


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0; 0.0 0.0 … 1.0 1.0], [103598.9592259127 2.6509912760219816 … 4.152052936007884 12.932370978171347; 80593.35 1.0 … 4.0 3.0; 308466.7458884541 3.559442903805858 … 4.084869317717121 33.11111086141694; 42062.26233024023 2.027187404955646 … 4.072225779248718 5.39066559487364], ConvergenceHistory("ProxGradGLRM", [Inf, 2.7418859153369528e14, 1.0421020974863781e14, 9.665882706972905e13, 9.457862082986962e13, 9.34858507238223e13, 9.16706005761707e13, 9.10457267248301e13, 9.078635838418202e13, 9.060266154724081e13, 9.04964065895266e13, 9.038951258974878e13, 9.032721554154312e13, 9.02663074716619e13, 9.023383540769802e13, 9.019872046047827e13, 9.019715729195572e13], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.34141111373901367, 1.661210060119629, 2.2585432529449463, 2.610260248184204, 2.956754

In [38]:
X4= PyReverseDims(X)

PyObject array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [27]:
X = convert(DataFrame, X)


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
CSV.write("../data/cancer4quad.csv", X)

In [16]:
PyReverseDims(X)

MethodError: MethodError: no method matching PyReverseDims(::DataFrame)
Closest candidates are:
  PyReverseDims(!Matched::Union{DenseArray{T,N}, Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, Base.ReshapedArray{T,N,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, SubArray{T,N,A,I,L} where L where I<:Tuple{Vararg{Union{Int64, AbstractRange{Int64}, Bas

In [11]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm, nfolds=4, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 134299 samples and testing on 44721
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 6.447025739614471e13
computing train and test error for fold 1:
	train error: 4.8001585198691386e8
	test error:  1.0935026047220385e9

forming train and test GLRM for fold 2
training model on 134167 samples and testing on 44853
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 7.172113023481761e13
computing train and test error for fold 2:
	train error: 5.34250441219968e8
	test error:  9.238132819729986e8

forming train and test GLRM for fold 3
training model on 134456 samples and testing on 44564
fitting train GLRM for fold 3
Fitting GLRM
Iteration 10: objective value = 7.740016365904039e13
Iteration 20: objective value = 7.711675833913516e13
computing train and test error for fold 3:
	train error: 5.72669

,train_error,test_error
,Float64,Float64
1,4.80016e8,1.0935e9
2,5.3425e8,9.23813e8
3,5.72669e8,9.08662e8
4,5.48331e8,8.91125e8


In [88]:
X,Y,ch = fit!(glrm)

Fitting GLRM
Iteration 10: objective value = 3.2805339662568237e12
Iteration 20: objective value = 3.224545614791724e12
Iteration 30: objective value = 3.2061817551191025e12


([0.0 0.0 … 0.0 1.0; 1.0 1.0 … 1.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [113184.78482324463 2.340246337498031 … 4.046487473627556 7.816767914895969; 44521.31827694036 1.8615761382422633 … 4.055640059452669 4.083095034249618; 327563.5215575475 3.6581914529729516 … 3.8908319495044776 31.39960870369608; 131185.96 2.0 … 3.0 2.0], ConvergenceHistory("ProxGradGLRM", [Inf, 8.536269346374443e12, 3.7486746107862393e12, 3.57768305812089e12, 3.5055349202241167e12, 3.4560368611355483e12, 3.371443760229454e12, 3.3327930253829106e12, 3.3113845539291714e12, 3.2927684809191685e12  …  3.213785822340414e12, 3.212441917284759e12, 3.211273764014428e12, 3.209844840137926e12, 3.2074742641460703e12, 3.2061817551191025e12, 3.205021153597181e12, 3.203401150015468e12, 3.2023527124434165e12, 3.202143962474778e12], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0,

In [89]:
X = convert(DataFrame, X)
CSV.write("../data/cancer4quad.csv", X)

"../data/cancer4quad.csv"

In [15]:
X,Y,ch = fit!(glrm)

Fitting GLRM
Iteration 10: objective value = 4.099376798805289e12


([1.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [33420.672901369675 1.7296703970331702 … 3.9557554235376706 3.2943347822477898; 166281.67292282914 2.528691892311409 … 4.054463378986292 12.51196033233945; 60571.32774550853 2.2075346372166855 … 4.06697679930551 6.385815075558724; 46216.84219269738 1.9583586454953394 … 4.218650486924203 4.1903747401096645], ConvergenceHistory("ProxGradGLRM", [Inf, 1.2551296796617248e13, 5.268198095228664e12, 4.81667220099305e12, 4.766552055125784e12, 4.71597316468396e12, 4.609425322287994e12, 4.463761913706175e12, 4.3056413543919062e12, 4.187344197951655e12, 4.099376798805289e12, 4.0536243317866626e12, 4.027974741832973e12, 4.0158800144847646e12, 4.009342796492845e12, 4.005777035124553e12, 4.004265402393816e12, 4.0032669656317686e12, 4.002921919973579e12], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [16]:
X = convert(DataFrame, X)
CSV.write("../data/cancer4quad.csv", X)

"../data/cancer4quad.csv"

In [17]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm, nfolds=4, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 6401 samples and testing on 2199
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 3.599753972016063e12
Iteration 20: objective value = 3.475273975832182e12
Iteration 30: objective value = 3.2768024711401123e12
Iteration 40: objective value = 3.175230403166264e12
Iteration 50: objective value = 3.1390001201005586e12
Iteration 60: objective value = 3.1163929119786914e12
computing train and test error for fold 1:
	train error: 4.8649362662906814e8
	test error:  1.6726007804486382e9

forming train and test GLRM for fold 2
training model on 6510 samples and testing on 2090
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 1.56294095290953e12
computing train and test error for fold 2:
	train error: 2.400792716797506e8
	test error:  1.337552856693221e9

forming train and test GLRM for fold 3
train

,train_error,test_error
,Float64,Float64
1,4.86494e8,1.6726e9
2,2.40079e8,1.33755e9
3,5.36657e8,4.64285e8
4,5.11042e8,5.59107e8


# kmeans k=4 different loss function

# function kmeans_proxgrad_drg(kval, lossarr, matrix)
    k = kval
    losses = lossarr
    A = matrix
    rx = UnitOneSparseConstraint()
    ry = ZeroReg()
    glrm = GLRM(A,losses,rx,ry,k)
    params = ProxGradParams(inner_iter_Y=40)

    
    X,Y,ch = fit!(glrm, params) # objective value
    println("X: ")
    println(X)
    println("Y: ")
    println(Y)
    
    imputey = impute(losses, Y) 
    imputey  = convert(DataFrame, imputey) 
    CSV.write(string("../data/iyproxgrad_drg", k,".csv" ), imputey)
    #CSV.read(string("../data/iywp", k,".csv" ), imputey)
    
    X= convert(DataFrame, X) 
    CSV.write(string("../data/Xproxgrad_drg", k,".csv" ), X)  
    #CSV.read(string("../data/Xwp", k,".csv" ), X)
#end   

In [21]:
loss1 = fill(HuberLoss(),1)
loss2 = fill(OrdinalHingeLoss(), 4)
#loss3 = fill(HingeLoss(), 1)

lossarr = [loss1; loss2]
#lossarr = [loss1; loss2; loss3]
lossarr

5-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))

In [22]:
k = 4
losses = lossarr
A = dfsmall1
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm = GLRM(A,losses,rx,ry,k)
#params = ProxGradParams(inner_iter_Y=40)

GLRM(1720×5 DataFrame. Omitted printing of 2 columns
│ Row  │ Total_Charges │ APR_Severity_of_Illness_Code │ APR_Risk_of_Mortality │
│      │ Float64       │ Int64                        │ Int64                 │
├──────┼───────────────┼──────────────────────────────┼───────────────────────┤
│ 1    │ 20601.5       │ 2                            │ 4                     │
│ 2    │ 12020.9       │ 1                            │ 1                     │
│ 3    │ 40220.4       │ 2                            │ 1                     │
│ 4    │ 9914.54       │ 1                            │ 1                     │
│ 5    │ 10182.1       │ 2                            │ 1                     │
│ 6    │ 9090.09       │ 2                            │ 1                     │
│ 7    │ 28436.7       │ 2                            │ 1                     │
│ 8    │ 6183.3        │ 2                            │ 1                     │
│ 9    │ 6342.01       │ 2                            │ 1          

In [23]:
X,Y,ch = fit!(glrm)

Fitting GLRM
Iteration 10: objective value = 9.4649984322356e7


([1.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [6.987612980934305 2.0020869585568164 … 3.985891233648444 6.005150177580298; 1.2694163334820323 0.665839135683542 … 3.4340923478011547 0.5124660683130968; 2.2739189060920486 0.7993599449773302 … 3.113246517544732 1.1468609967280443; -0.5246299600835251 -1.287753696024506 … -0.2175747033212368 -0.9164988468666255], ConvergenceHistory("ProxGradGLRM", [Inf, 9.468647765952681e7, 9.466267439259824e7, 9.46599611098231e7, 9.465803589752518e7, 9.465686488577506e7, 9.465548043027258e7, 9.465405129052033e7, 9.465270709073469e7, 9.465138563019902e7, 9.4649984322356e7, 9.464855695195684e7], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.08644390106201172, 0.23240900039672852, 0.34013986587524414, 0.3681628704071045, 0.3975660800933838, 0.4272000789642334, 0.45972394943237305, 0.4913458824157715, 0.5487518310546875, 0.57694888

In [24]:
X = convert(DataFrame, X)
CSV.write("../data/cancer4diff.csv", X)

"../data/cancer4diff.csv"

In [25]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm, nfolds=4, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 6523 samples and testing on 2077
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 6.971489601014723e7
computing train and test error for fold 1:
	train error: 10687.341305861837
	test error:  11998.11923212964

forming train and test GLRM for fold 2
training model on 6466 samples and testing on 2134
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 7.149885235728997e7
computing train and test error for fold 2:
	train error: 11057.455935789529
	test error:  10842.052596336698

forming train and test GLRM for fold 3
training model on 6415 samples and testing on 2185
fitting train GLRM for fold 3
Fitting GLRM
Iteration 10: objective value = 7.101043895282283e7
computing train and test error for fold 3:
	train error: 11069.227849638475
	test error:  10812.32274463921

forming train and test GLR

,train_error,test_error
,Float64,Float64
1,10687.3,11998.1
2,11057.5,10842.1
3,11069.2,10812.3
4,11206.0,10417.7


In [ ]:
#init kmean demo
init_kmeanspp!(glrm)
X,Y,ch = fit!(glrm)